In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
browser = webdriver.Chrome()
browser.get('https://kinrou.sas-cloud.jp/kinrou/kojin/')

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 78


In [ ]:
elem_com = browser.find_element_by_id('houjin_code')
elem_com.send_keys('10338')
elem_worker = browser.find_element_by_id('user_id')
elem_worker.send_keys('0000014647')
elem_pass = browser.find_element_by_id('password')
elem_pass.send_keys('kino@0120')
elem_btn = browser.find_element_by_id('bt')
elem_btn.click()

In [ ]:
from selenium.webdriver.support.ui import Select
select_month_element = browser.find_element_by_name('selectedMonth')
select_year_element = browser.find_element_by_name('selectedYear')
select_month_element = Select(select_month_element)
select_year_element.clear()
select_year_element.send_keys('2020')
select_month_element.select_by_value('02')
elem_btn = browser.find_element_by_name('changeGetudo')
elem_btn.click()

In [ ]:
elem_btn2 = browser.find_elements_by_id('popover')
elem_btn2[1].click()

In [ ]:
work_time_table = browser.find_element_by_id('id_data_table_body')

In [ ]:
table_row = work_time_table.find_elements_by_class_name('cell_height_size')

In [ ]:
df1 = []
for row in table_row:
    row_items = row.find_elements_by_tag_name('td')
    row_items_text = [item.text for item in row_items]
    df1.append(row_items_text)
df1 =  pd.DataFrame(df1)

In [ ]:
TrueCol = [23,24,25,27]
df1 = df1.iloc[:,TrueCol]
df1.columns = ['残業時間', '深夜時間', '休日出勤', '基本時間']

In [ ]:
df2 = []
targets = ['_1_47', '_1_48', '_2_59']
for index, row in enumerate(table_row):
    row_items = [row.find_elements_by_name('dataList[{}].column{}'.format(index,target)) for target in targets]
    row_items_text = [item[0].get_attribute("value") for item in row_items]
    df2.append(row_items_text)
df2 = pd.DataFrame(df2)

In [ ]:
df2.columns = ['作業開始', '作業終了', '休憩時間']

In [ ]:
df_time = pd.concat([df2, df1], axis=1)
df_time = df_time.loc[:,['作業開始','作業終了','休憩時間','基本時間','残業時間','深夜時間','休日出勤']]

In [ ]:
date_table = browser.find_element_by_id('id_hidari_kotei_body')
date_table_row = date_table.find_elements_by_class_name('cell_height_size')
df3 = []
for num, row in enumerate(date_table_row):
    row_items = row.find_elements_by_tag_name('td')
    row_items_text = [item.text for item in row_items]
    df3.append(row_items_text)
#     target = date_table_row[num]
#     actions = ActionChains(browser)
#     actions.move_to_element(target)
#     actions.perform()
df3 =  pd.DataFrame(df3)

#途中までしか拾えないので、一番下までスクロールしてからもう一度データを収拾する
browser.find_element_by_tag_name('body').send_keys(Keys.END)

date_table_row = date_table.find_elements_by_class_name('cell_height_size')
df4 = []
for num, row in enumerate(date_table_row):
    row_items = row.find_elements_by_tag_name('td')
    row_items_text = [item.text for item in row_items]
    df4.append(row_items_text)
df4 =  pd.DataFrame(df4)

df_date = pd.concat([df3.iloc[:15], df4.iloc[15:]])
print(df_date)
browser.close()

In [ ]:
df_date = df_date.iloc[:, [3,4]]
df_date.columns = ['日', '曜']
print(df_date)

In [ ]:
#df_date.iloc[:,0]　= [for i.split('月')[1] in df_date.iloc[:,1]]
#df_date['日']=[i.split('/')[1] for i in df_date['日']]

In [ ]:
df_all = pd.concat([df_date, df_time], axis = 1)
df_all.to_csv('業務日報_data.csv',encoding='utf-8-sig')